In [1]:
end_date = "2017-09-20"
start_date = "2017-07-18"

Save_Path='/Users/JayLiang/Desktop/Media Storm/PEI/Perry Ellis/SEM Optimization/Non_Brand_PE'+'_From_'+str(start_date)+'_To_'+str(end_date)+'.xlsx'
print(Save_Path)

import pandas as pd
import numpy as np
pd.options.display.max_rows = 6
import os
import argparse
import httplib2
import pprint
import time
import datetime
from io import StringIO

from apiclient.discovery import build
from oauth2client import GOOGLE_TOKEN_URI
from oauth2client.client import OAuth2Credentials
from googleapiclient.errors import HttpError


def create_credentials():
    """Create Google OAuth2 credentials.

    Args:
        client_id: Client id of a Google Cloud console project.
        client_secret: Client secret of a Google Cloud console project.
        refresh_token: A refresh token authorizing the Google Cloud console project
          to access the DS data of some Google user.
    Returns:
        OAuth2Credentials
    """
    return OAuth2Credentials(access_token=None,
                           client_id='549790627766-qnth4m8qvuimg87pnsp4b82lhte7dk5a.apps.googleusercontent.com',
                           client_secret='Vta4lQLOL49vVYvktkcPGRNb',
                           refresh_token='1/ab7pCGMu3K5AveG0UOUpQ0J08vCp6uM357O8qmoPDMs',
                           token_expiry=None,
                           token_uri="https://accounts.google.com/o/oauth2/token",
                           user_agent=None)

def get_service(credentials):
    """Set up a new DoubleClick Search service.

    Args:
        credentials: An OAuth2Credentials generated with create_credentials, or
        flows in the oatuh2client.client package.
    Returns:
        An authorized Doubleclicksearch serivce.
    """
    # Use the authorize() function of OAuth2Credentials to apply necessary credential
    # headers to all requests.
    http = credentials.authorize(http = httplib2.Http())

    # Construct the service object for the interacting with the DoubleClick Search API.
    service = build('doubleclicksearch', 'v2', http=http)
    return service

def poll_report(service, report_id):
    """Poll the API with the reportId until the report is ready, up to ten times.

    Args:
        service: An authorized Doublelcicksearch service.
        report_id: The ID DS has assigned to a report.
    Returns:
        pd.DataFrame, report file
    """
    for _ in range(10):
        try:
            request = service.reports().get(reportId=report_id)
            json_data = request.execute()
            if json_data['isReportReady']:
                pprint.pprint('The report is ready.')

                # For large reports, DS automatically fragments the report into multiple
                # files. The 'files' property in the JSON object that DS returns contains
                # the list of URLs for file fragment. To download a report, DS needs to
                # know the report ID and the index of a file fragment.
                report = pd.DataFrame()
                for i in range(len(json_data['files'])):
                    pprint.pprint('Downloading fragment ' + str(i) + ' for report ' + report_id)
                    report = report.append(download_files(service, report_id, str(i)), ignore_index = True) # See Download the report.
                return report

            else:
                pprint.pprint('Report is not ready. I will try again.')
                time.sleep(10)
        except HttpError as e:
            error = simplejson.loads(e.content)['error']['errors'][0]

            # See Response Codes
            pprint.pprint('HTTP code %d, reason %s' % (e.resp.status, error['reason']))
            break
        
def download_files(service, report_id, report_fragment):
    """Generate and print sample report.

    Args:
        service: An authorized Doublelcicksearch service.
        report_id: The ID DS has assigned to a report.
        report_fragment: The 0-based index of the file fragment from the files array.
    Returns:
        pd.DataFrame report file
    """
    request = service.reports().getFile(reportId=report_id, reportFragment=report_fragment)
    return pd.read_csv(StringIO(request.execute().decode('utf-8')))

def request_report(service, start_date, end_date, columns):
    """Request sample report and print the report ID that DS returns. See Set Up Your Application.

    Args:
        service: An authorized Doublelcicksearch service.
        columns: list of columns will be in the report
    Returns:
        The report id.
    """
    request = service.reports().request(
        body={
                "reportScope": {
                    "agencyId": "20100000000000932",
                    "advertiserId": "21700000001423776", # Callaway Apparel - Perry Ellis International
                    #"engineAccountId": "700000001564770" # Google - Callaway Apparel
                    #"advertiserId": "21700000001131725", # Celebrity Cruise
                    #"engineAccountId": "700000001217833" # Celebrity Cruise
                    #"engineAccountId": "700000001561242" # Celebrity Cruise - Juba Plus
                },
                "reportType": "keyword",
                "columns": [{'columnName': column} for column in columns],   
                "timeRange" : {
                    "startDate" : start_date,
                    "endDate" : end_date
                    },
                
                #"filters": [
                #    {
                #        "column" : { "columnName": "keywordLabels" },
                #        "operator" : "containsElement",
                #        "values" : ["JubaNovTest",]
                #    }
                #],
                
                "downloadFormat": "csv",
                "maxRowsPerFile": 100000000,
                "statisticsCurrency": "agency",
                "verifySingleTimeZone": "false",
                "includeRemovedEntities": "false"
            }
    )
    json_data = request.execute()
    return json_data['id']

/Users/JayLiang/Desktop/Media Storm/PEI/Perry Ellis/SEM Optimization/Non_Brand_PE_From_2017-07-18_To_2017-09-20.xlsx


In [2]:
# download reports
creds = create_credentials()

service = get_service(creds)


print(start_date, end_date)
REPORTID_nonHVA = request_report(service, start_date, end_date, 
                                 ['campaign', 'adGroup', 'keywordText', 'keywordMatchType', 'status', 
                                  'effectiveKeywordMaxCpc', 'keywordMaxCpc', 'topOfPageBidCurrent',
                                  'topOfPageBidAvg', 'impr', 'clicks', 'cost', 
                                  'avgCpc', 'avgPos', 'dfaRevenue'])
REPORTID_HVA = request_report(service, start_date, end_date, 
                              ['campaign', 'adGroup', 'keywordText', 'keywordMatchType', 
                               'floodlightActivity', 'dfaActions'])

non_hva = poll_report(service, REPORTID_nonHVA)
hva = poll_report(service, REPORTID_HVA)

2017-07-18 2017-09-20
'Report is not ready. I will try again.'
'The report is ready.'
'Downloading fragment 0 for report AAAnlwTMxh9biVUx'
'The report is ready.'
'Downloading fragment 0 for report AAAntHWumGePHsVo'


# merge reports

In [3]:
#Copy and include all
def merge_hva_and_non_hva(hva, non_hva):
    '''merge two reports downloaded by download_reports().
    Args:
        hva: pd.DataFrame
        non_hva: pd.DataFrame
        columns_hva: list of string
    Returns:
        pd.DataFrame
    '''   
    columns_hva= ['Perry Ellis - Store Locator',
 'Perry Ellis - Supreme Perks',
 'Perry Ellis - About Us',
 'Perry Ellis - Accessories_Accessories',
 'Perry Ellis - Accessories_Accessories_Athletic-Socks',
 'Perry Ellis - Accessories_Accessories_Bags_Luggage',
 'Perry Ellis - Accessories_Accessories_Belts',
 'Perry Ellis - Accessories_Accessories_Cufflinks',
 'Perry Ellis - Accessories_Accessories_Dress-Socks',
 'Perry Ellis - Accessories_Accessories_Grooming-Kits',
 'Perry Ellis - Accessories_Accessories_Hats',
 'Perry Ellis - Accessories_Accessories_Suiting-Essentials',
 'Perry Ellis - Accessories_Accessories_Sunglasses',
 'Perry Ellis - Accessories_Accessories_Ties_Pocket-Squares',
 'Perry Ellis - Accessories_Accessories_Wallets',
 'Perry Ellis - Accessories_Accessories_Watches',
 'Perry Ellis - Accessories_Fragrance',
 'Perry Ellis - Accessories_Fragrance_Gift-Sets',
 'Perry Ellis - Accessories_Fragrance_Mens-Fragrance',
 'Perry Ellis - Accessories_Fragrance_Womens-Fragrance',
 'Perry Ellis - Accessories_Shoes',
 'Perry Ellis - Accessories_Shoes_Casual-Shoes',
 'Perry Ellis - Accessories_Shoes_Dress-Shoes',
 'Perry Ellis - Big-Tall_Casual-Shirts',
 'Perry Ellis - Big-Tall_New-Arrivals',
 'Perry Ellis - Big-Tall_Outerwear'#Not Found',
 'Perry Ellis - Big-Tall_Pants',
 'Perry Ellis - Big-Tall_Polos-Knits',
 'Perry Ellis - Big-Tall_Sale'#Not Found',
 'Perry Ellis - Big-Tall_Suits',
 'Perry Ellis - Clothing_Features_360-Performance-Collection',
 'Perry Ellis - Clothing_Features_The-Slim-Shop',
 'Perry Ellis - Clothing_Features_The-Travel-Luxe-Collection',
 'Perry Ellis - Clothing_Pants',
 'Perry Ellis - Clothing_Pants_Casual Pants',
 'Perry Ellis - Clothing_Pants_Denim',
 'Perry Ellis - Clothing_Pants_Dress-Pants',
 'Perry Ellis - Clothing_Pants_Shorts',
 'Perry Ellis - Clothing_Suit-Separates_Jackets',
 'Perry Ellis - Clothing_Suit-Separates_Pants',
 'Perry Ellis - Clothing_Suit-Separates_Suits',
 'Perry Ellis - Clothing_Suit-Separates_Vests',
 'Perry Ellis - Clothing_Tops',
 'Perry Ellis - Clothing_Tops_Casual-Shirts',
 'Perry Ellis - Clothing_Tops_Dress-Shirts',
 'Perry Ellis - Clothing_Tops_Outerwear',
 'Perry Ellis - Clothing_Tops_Polo-Knits',
 'Perry Ellis - Clothing_Tops_Sweaters',
 'Perry Ellis - Products - Bags',
 'Perry Ellis - Products - Belts',
 'Perry Ellis - Products - Casual-Pants',
 'Perry Ellis - Products - Casual-Shirts',
 'Perry Ellis - Products - Cologne',
 'Perry Ellis - Products - Cufflinks',
 'Perry Ellis - Products - Denim',
 'Perry Ellis - Products - Dress-Pants',
 'Perry Ellis - Products - Dress-Shirts',
 'Perry Ellis - Products - Gifables',
 'Perry Ellis - Products - Hats',
 'Perry Ellis - Products - Jackets-Outerwear',
 'Perry Ellis - Products - Perfume',
 'Perry Ellis - Products - Polo-Knits',
 'Perry Ellis - Products - Shoes',
 'Perry Ellis - Products - Shorts',
 'Perry Ellis - Products - Sleepwear',
 'Perry Ellis - Products - Socks',
 'Perry Ellis - Products - Suit-Jackets',
 'Perry Ellis - Products - Suit-Vests',
 'Perry Ellis - Products - Sunglasses',
 'Perry Ellis - Products - Sweaters',
 'Perry Ellis - Products - Ties',
 'Perry Ellis - Products - Underwear',
 'Perry Ellis - Products - Wallets',
 'Perry Ellis - Products - Watches',
 'Perry Ellis - Sale_Accessories',
 'Perry Ellis - Sale_Big-and-Tall',
 'Perry Ellis - Search',
 'Perry Ellis - Underwear',
 'Perry Ellis - Underwear_Boxer-Briefs',
 'Perry Ellis - Underwear_Boxers',
 'Perry Ellis - Underwear_Multipacks',
 'Perry Ellis - Underwear_Sleepwear',
 'Perry Ellis - Email Sign Up Thank You Page',
 'Perry Ellis - Order Review - All',
 'Perry Ellis - PayPal Checkout',
 'Perry Ellis - Secure Checkout'#Not Found',
 'Perry Ellis - Shipping - All',
 'Perry Ellis - Billing Payment - All',
 'Perry Ellis - Thank You Page - Items Sold'#Not Found',
 'Perry Ellis - Thank You Page - Transactions'#Not Found (Perry Ellis - Thank You Page - Transactions, for not use)',
 'Perry Ellis - Add to Cart']
    
    result = pd.DataFrame(columns=['campaign', 'adGroup', 'keywordText', 'keywordMatchType']+columns_hva)
    
    for (campaign, ad_group, keyword, keyword_match_type), group in hva.groupby(['campaign', 'adGroup', 
                                                                                 'keywordText', 'keywordMatchType']):
        df = pd.DataFrame([{
            'campaign': campaign,
            'adGroup': ad_group,
            'keywordText' : keyword,
            'keywordMatchType': keyword_match_type
        }])

        for column in columns_hva:
            if column in group['floodlightActivity'].values:
                df[column] = group[group['floodlightActivity'] == column]['dfaActions'].values[0]
            else:
                df[column] = 0
                
        result = result.append(df, ignore_index = True)

    # combine hva and non_hva
    merged = non_hva.merge(result, 
                           on = ['campaign', 'adGroup', 'keywordText', 'keywordMatchType'], 
                           how = 'left')

    # generate baseline and resid compare
    merged = merged.fillna(value = 0)
    
    # generate new fields
    merged['HVA'] = merged[columns_hva].sum(axis=1).apply(int)   
    merged['ROI'] = merged['dfaRevenue'] / merged['cost']
    merged['ROI'] = merged['ROI'].fillna(0)
    return merged[['campaign', 'adGroup', 'keywordText', 'keywordMatchType', 
                   'status', 'keywordMaxCpc', 'effectiveKeywordMaxCpc', 
                   'topOfPageBidCurrent', 'topOfPageBidAvg', 'impr', 'clicks', 
                   'HVA', 'avgCpc', 'avgPos', 'cost', 'dfaRevenue', 'ROI']]

In [4]:
writer = pd.ExcelWriter(Save_Path, engine='xlsxwriter')
campaigns = {
    'G_Non-Brand BigTall': 'G_Non-Brand_Perry Ellis_Big & Tall_EST-2777',
    'G_Non-Brand Men': 'G_Non-Brand_Perry Ellis_Mens_EST-2777',
    'B_Non-Brand BigTall': 'B_Non-Brand_Perry Ellis_Big & Tall_EST-2777',
    'B_Non-Brand Men': 'B_Non-Brand_Perry Ellis_Mens_EST-2777',
    'Y_Non-Brand BigTall': 'Y_Non-Brand_Perry Ellis_Big & Tall_EST-2777',
    'Y_Non-Brand Men': 'Y_Non-Brand_Perry Ellis_Mens_EST-2777'
}

List=[]
for tab in campaigns:
    CAMPAIGN = campaigns[tab]
    hva_1 = hva[hva['campaign'] == CAMPAIGN]
    non_hva_1 = non_hva[non_hva['campaign'] == CAMPAIGN]
    print(CAMPAIGN, len(hva_1))
    df = merge_hva_and_non_hva(hva_1, non_hva_1)
    df = df[(df['keywordText'] != 'Display Network Stats') & (df['status'] == 'Active')]
    if len(df)==0:
        df['group'] = None

    elif len(df['ROI'].unique())==1 and df['ROI'].unique()[0]==0 and len(df['HVA'].unique())==1 and df['HVA'].unique()[0]==0:
        df['group'] = None

    else:
        # group1
        df.loc[(df['ROI'] > 0)&(df['clicks'] < 30), 'group'] = 'group1_lowClick'
        df.loc[(df['ROI'] > 0)&(df['clicks'] >= 30), 'group'] = 'group1'

        # group2
        HVA_mean = df['HVA'].mean()
        if HVA_mean < 10:
            impr_75 = sorted(list(df['impr']))
            HVA_mean = df[df['HVA'] > 0]['HVA'].mean()
        impr_75 = sorted(list(df['impr']))[int(np.floor(int(len(df)*3/4)))]

        df.loc[(df['ROI'] == 0)&
               (df['HVA'] >= HVA_mean)&
               (df['clicks'] >= 30)&
               (df['impr']>=impr_75), 'group'] = 'group2'
        df.loc[(df['ROI'] == 0)&
               (df['HVA'] >= HVA_mean)&
               (df['clicks'] >= 30)&
               (df['impr']<impr_75), 'group'] = 'group2_highImpr'
        df.loc[(df['ROI'] == 0)&
               (df['HVA'] >= HVA_mean)&
               (df['clicks'] < 30), 'group'] = 'group2_lowClick'

        impr_mean = df[df['group'].isnull()]['impr'].mean() #Not allocated group 'impr' mean

        # group3
        df.loc[(df['ROI'] == 0)&
               (df['HVA'] < HVA_mean)&
               (df['impr'] >= impr_mean)&
               (df['cost'] > 0)&
               (df['clicks'] >= 30), 'group'] = 'group3'
        df.loc[(df['ROI'] == 0)&
               (df['HVA'] < HVA_mean)&
               (df['impr'] >= impr_mean)&
               (df['cost'] > 0)&
               (df['clicks'] < 30), 'group'] = 'group3_lowClick'

        # group4
        df.loc[(df['ROI'] == 0)&
               (df['HVA'] < HVA_mean)&
               (df['impr'] >= impr_mean)&
               (df['avgPos'] <= 2)&
               (df['cost'] == 0), 'group'] = 'group4'
        df.loc[(df['ROI'] == 0)&
               (df['HVA'] < HVA_mean)&
               (df['impr'] >= impr_mean)&
               (df['avgPos'] > 2)&
               (df['cost'] == 0), 'group'] = 'group4_lowPosition'

        # group5
        df.loc[(df['ROI'] == 0)&
               (df['HVA'] < HVA_mean)&
               (df['impr'] < impr_mean)&
               (df['clicks'] >= 30), 'group'] = 'group5'
        df.loc[(df['ROI'] == 0)&
               (df['HVA'] < HVA_mean)&
               (df['impr'] < impr_mean)&
               (df['clicks'] < 30), 'group'] = 'group5_lowClick'




        applications_SEM = {
            'group1': 'Maintain',
            'group1_lowClick': 'keep_running',
            'group2': '75% Desktop, 25% Mobile',
            'group2_highImpr': 'Type= Exact, add ngatiove KWs',
            'group2_lowClick': 'keep_running',
            'group3': 'pause/ Add to sitelinkes in other ad-groups',
            'group3_lowClick': 'keep_running',
            'group4': 'keep_running',
            'group4_lowPosition': 'improve position to 1-2 (increase CPC)',
            'group5': 'pause',
            'group5_lowClick': 'keep_running'}

        applications_FB_Programatic = {
            'group1': 'Add Products',
            'group1_lowClick': 'keep_running',
            'group2': 'Add Products',
            'group2_highImpr': 'Add Products',
            'group2_lowClick': 'keep_running',
            'group3': 'n.a',
            'group3_lowClick': 'keep_running',
            'group4': 'n.a',
            'group4_lowPosition': 'n.a',
            'group5': 'n.a',
            'group5_lowClick': 'keep_running'}

        applications_PLA = {
            'group1': 'Maintain/add Products',
            'group1_lowClick': 'keep_running',
            'group2': 'Filter by Zips',
            'group2_highImpr': 'Add Products',
            'group2_lowClick': 'keep_running',
            'group3': 'pause',
            'group3_lowClick': 'keep_running',
            'group4': 'keep running',
            'group4_lowPosition': 'improve position to 1-2 (increase CPC)',
            'group5': 'n.a',
            'group5_lowClick': 'keep_running'}
        applications_Display = {
            'group1': 'Add Products',
            'group1_lowClick': 'keep_running',
            'group2': 'Filter by Zips/100% Desktop',
            'group2_highImpr': 'n.a',
            'group2_lowClick': 'keep_running',
            'group3': 'n.a.',
            'group3_lowClick': 'keep_running',
            'group4': 'n.a',
            'group4_lowPosition': 'n.a',
            'group5': 'n.a',
            'group5_lowClick': 'keep_running'}
        criterion = {
            'group1': 'ROI > 0',
            'group1_lowClick': 'clicks are too low to drive conclusion',
            'group2': 'ROI = 0, high HVA',
            'group2_highImpr': 'ROI = 0, high HVA, high impression',
            'group2_lowClick': 'clicks are too low to drive conclusion',
            'group3': 'ROI = 0, low HVA, high impression, cost > 0',
            'group3_lowClick': 'clicks are too low to drive conclusion',
            'group4': 'ROI = 0, low HVA, high impression, cost = 0, position already high',
            'group4_lowPosition': 'ROI = 0, low HVA, high impression, cost = 0, low position',
            'group5': 'ROI = 0, low HVA, low impression',
            'group5_lowClick': 'clicks are too low to drive conclusion'}

        df['criterion'] = df['group'].apply(lambda x: criterion[x])
        df['applications_FB_Programatic'] = df['group'].apply(lambda x: applications_FB_Programatic[x])
        df['applications_SEM'] = df['group'].apply(lambda x: applications_SEM[x])
        df['applications_PLA'] = df['group'].apply(lambda x: applications_PLA[x])
        df['applications_Display'] = df['group'].apply(lambda x: applications_Display[x])
    
    df.sort_values('group', inplace=True)
    List.append(df)
    df.to_excel(writer, sheet_name=tab, index=False)


#####

new_df = []
for i in range(len(List)):
    if i%3 == 0:
        newdf = pd.concat([List[i],List[i+1],List[i+2]])
        new_df.append(newdf)

newnew_df = []        
for summ in new_df:
    summ['Engine']=summ['campaign'].apply(lambda x: x[0])
    summ = pd.pivot_table(summ[['campaign','Engine','group','keywordText']],index = ['campaign','Engine','group'], aggfunc = 'count' )
    summ.columns =summ.columns.get_level_values(0)
    summ.reset_index(inplace=True)
    summ['SEM RECOMMENDATION'] = summ['group'].apply(lambda x : applications_SEM[x])
    summ['FB/PROGRAMMATIC'] = summ['group'].apply(lambda x : applications_FB_Programatic[x])
    summ['PLA RECOMMENDATION'] = summ['group'].apply(lambda x : applications_PLA[x])
    summ['Display RECOMMENDATION'] = summ['group'].apply(lambda x : applications_Display[x])
    newnew_df.append(summ)
    
summary = pd.concat(newnew_df)
summary_2=summary.pivot_table(summary[['campaign','Engine','SEM RECOMMENDATION','FB/PROGRAMMATIC','PLA RECOMMENDATION','Display RECOMMENDATION','keywordText']],index = ['campaign','Engine','SEM RECOMMENDATION','FB/PROGRAMMATIC','PLA RECOMMENDATION','Display RECOMMENDATION'], aggfunc = 'sum' )
summary_2.columns=summary_2.columns.get_level_values(0)
summary_2.reset_index(inplace=True)
summary_2['campaign']=summary_2['campaign'].apply(lambda x: x[24:(len(x)-9)])
summary_2.sort_values(['campaign','Engine'], inplace=True)
summary_2=summary_2[['campaign', 'Engine', 'keywordText', 'SEM RECOMMENDATION', 'FB/PROGRAMMATIC',
       'PLA RECOMMENDATION', 'Display RECOMMENDATION']]
Engine_Rep_dict={'G':"Google","B":"Bing","Y":'Yahoo'}
summary_2['Engine']=summary_2['Engine'].apply(lambda x:Engine_Rep_dict[x])
summary_2.to_excel(writer, sheet_name='summary', index=False)


writer.save()

G_Non-Brand_Perry Ellis_Big & Tall_EST-2777 2958
G_Non-Brand_Perry Ellis_Mens_EST-2777 2674
B_Non-Brand_Perry Ellis_Big & Tall_EST-2777 2826
B_Non-Brand_Perry Ellis_Mens_EST-2777 2731
Y_Non-Brand_Perry Ellis_Big & Tall_EST-2777 2694
Y_Non-Brand_Perry Ellis_Mens_EST-2777 2220
